<a href="https://colab.research.google.com/github/hargurjeet/Adhoc-Activities/blob/main/Fake_review_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt # plotting
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as py
from wordcloud import WordCloud

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
import zipfile

def unzip_file(zip_file_path, output_dir):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)

# Example usage:
zip_file_path = '/content/ReviewerNamewise.zip'  # Replace this with the path to your zip file
output_dir = '/content/'  # Replace this with the directory where you want to extract the files
unzip_file(zip_file_path, output_dir)

In [ ]:
import os
import pandas as pd

def read_and_append_csv_files(folder_path):
    all_data = pd.DataFrame()  # Initialize an empty DataFrame to store the combined data

    # List all files in the folder
    file_list = os.listdir(folder_path)

    # Loop through each file
    for file_name in file_list:
        if file_name.endswith('.csv'):  # Check if the file is a CSV file
            file_path = os.path.join(folder_path, file_name)  # Get the full file path
            data = pd.read_csv(file_path)  # Read the CSV file
            all_data = all_data.append(data, ignore_index=True)  # Append the data to the DataFrame

    return all_data

# Example usage:
folder_path = '/content/ReviewerNamewise/NormalCase'  # Replace this with the path to your folder containing CSV files
combined_data = read_and_append_csv_files(folder_path)


In [5]:
combined_data.shape

(644, 5)

In [16]:
from joblib import load

# Load the model from the file
loaded_model = load('/content/drive/MyDrive/Documents/models.joblib')

In [15]:
loaded_model

[XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=2, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               n_estimators=100, n_jobs=None, num_parallel_tree=None,
               predictor=None, random_state=None, ...),
 RandomForestClassifier()]

In [8]:
import string
# Define a function to preprocess the review text
def preprocess_text(text):
    # Convert the text to lowercase and split it into words
    words = text.lower().split()

    # Remove punctuation and numbers from the words
    words = [word.translate(str.maketrans('', '', string.punctuation + string.digits)) for word in words]

    # Remove stop words from the words
    stopwords = set(nltk.corpus.stopwords.words('english'))
    words = [word for word in words if word not in stopwords]

    # Stem the words using a Porter stemmer
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # Return the preprocessed text as a list of words
    return ' '.join(words)

In [9]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

def get_word2vec_embedding(text):
    # Tokenize the text into words
    words = word_tokenize(text)

    # Train a word2vec model on the tokenized words
    model = Word2Vec([words], min_count=1, vector_size=100)

    # Get the word2vec embeddings for the words
    word_vectors = model.wv

    # Return the word2vec embeddings
    return word_vectors

In [22]:
new_comment="This is an example sentence."
pre_new_comment = preprocess_text(new_comment)
embeddings = get_word2vec_embedding(pre_new_comment)

# Data type of array element
Data_type = float

# This cause Value error
np_array = np.array(embeddings, dtype=Data_type)


# embeddings = np.reshape(embeddings, (1, -1))
# Predict the class label for the new record
# predicted_class = classifiers[-1].predict(embeddings)
predicted_class = loaded_model[-1].predict(np_array)
# Print the predicted class
print(predicted_class)

ValueError: ignored